In [1]:
###  INPUT ###

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.optimizers import SGD,Adam,RMSprop

input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

arr= ['DBSF1059','DBSF1093','DBSF1172','DBSF1180','DBSF1279','DBSF1294','DBSF1385','DBSF1459','DBSF1469','DBSF1507','DBSF1539','DBSF1601','DBSF1621','DBSF1871']

input_data['date']= input_data['date'].astype(str)
input_data["year"]=input_data["date"].str[:4]
input_data["month"]=input_data["date"].str[4:6]
input_data["day"]=input_data["date"].str[6:]
input_data['date']=input_data["year"]+'-'+input_data["month"]+'-'+input_data["day"]
input_data=input_data.drop(['month','year','day'],axis=1)
input_data['date']=pd.to_datetime(input_data['date'])
input_data=input_data.set_index(keys=['date'],inplace=False,drop=True)


In [2]:
input_data = input_data[input_data['HeatingEnergyUsage_cumsum'] != 0.000]
input_data = input_data[input_data['outtrn_cumsum'] != 0]

In [4]:
data1 = input_data[input_data['frmDist'] == arr[0]]
for i in range(1,len(arr)):
    data = input_data[input_data['frmDist'] == arr[i]]
    tmp=data
    data_result=pd.concat([data1,tmp])
    data1= data_result

In [5]:
data = data_result[['WaterUsage','FertilizerCost','ec','frtstCo','fcluHg']]
target =data_result[['HeatingEnergyUsage_cumsum','outtrn_cumsum']]

In [7]:
data

,WaterUsage,FertilizerCost,ec,frtstCo,fcluHg
date,,,,,
2022-11-16,1554.030854,5386.836093,3.561106,0.000000,0.000000
2022-11-18,1345.400471,5423.110788,2.696950,0.000000,0.000000
2022-11-21,1579.231214,4527.027519,2.554448,21.414553,17.184046
2022-11-23,1320.575421,5299.474480,2.195207,0.000000,0.000000
2022-11-25,1277.784333,4120.963356,2.270167,0.000000,0.000000
...,...,...,...,...,...
2022-11-30,1555.135470,4491.168479,2.020918,0.000000,0.000000
2022-12-02,1799.931639,4423.105334,2.257751,0.000000,0.000000
2022-12-05,1407.089056,4360.751475,2.225832,15.787118,26.590393


In [8]:
scaler = MinMaxScaler()
scale_cols =0
scale_cols = ['WaterUsage','FertilizerCost','ec','frtstCo','fcluHg']
scaled = scaler.fit_transform(data[scale_cols])
df_min_max= pd.DataFrame(scaled, columns = ['WaterUsage','FertilizerCost','ec','frtstCo','fcluHg'])
df_min_max.index = data.index
data=df_min_max

In [9]:
scale_cols = ['outtrn_cumsum','HeatingEnergyUsage_cumsum']
scaled = scaler.fit_transform(target[scale_cols])
df_min_max= pd.DataFrame(scaled, columns = ['outtrn_cumsum','HeatingEnergyUsage_cumsum'])
df_min_max.index = target.index
target=df_min_max

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2,shuffle=False)

In [13]:
aDam = keras.optimizers.Adam(lr=0.01)

C:\Users\USER\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [84]:
model = Sequential()
model.add(Dense(32,input_dim=5, activation='relu'))
model.add(Dense(16,activation='sigmoid'))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='mse', optimizer=aDam, metrics='mse')
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 32)                192       
                                                                 
 dense_49 (Dense)            (None, 16)                528       
                                                                 
 dense_50 (Dense)            (None, 2)                 34        
                                                                 
Total params: 754
Trainable params: 754
Non-trainable params: 0
_________________________________________________________________


In [85]:
model.fit(X_train,y_train, epochs=300,batch_size=32, verbose=1)

Epoch 1/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0715 - mse: 0.0715
Epoch 2/300
4/4 [==============================] - 0s 667us/step - loss: 0.0680 - mse: 0.0680
Epoch 3/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0573 - mse: 0.0573
Epoch 4/300
4/4 [==============================] - 0s 667us/step - loss: 0.0479 - mse: 0.0479
Epoch 5/300
4/4 [==============================] - 0s 667us/step - loss: 0.0403 - mse: 0.0403
Epoch 6/300
4/4 [==============================] - 0s 667us/step - loss: 0.0360 - mse: 0.0360
Epoch 7/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0337 - mse: 0.0337
Epoch 8/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0330 - mse: 0.0330
Epoch 9/300
4/4 [==============================] - 0s 1000us/step - loss: 0.0344 - mse: 0.0344
Epoch 10/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0321 - mse: 0.0321
Epoch 11/300
4/4 [==============================] - 0s 1ms/step - lo

4/4 [==============================] - 0s 667us/step - loss: 0.0219 - mse: 0.0219
Epoch 89/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222
Epoch 90/300
4/4 [==============================] - 0s 667us/step - loss: 0.0225 - mse: 0.0225
Epoch 91/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0225 - mse: 0.0225
Epoch 92/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0219 - mse: 0.0219
Epoch 93/300
4/4 [==============================] - 0s 667us/step - loss: 0.0219 - mse: 0.0219
Epoch 94/300
4/4 [==============================] - 0s 667us/step - loss: 0.0219 - mse: 0.0219
Epoch 95/300
4/4 [==============================] - 0s 667us/step - loss: 0.0223 - mse: 0.0223
Epoch 96/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0216 - mse: 0.0216
Epoch 97/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0217 - mse: 0.0217
Epoch 98/300
4/4 [==============================] - 0s 1000us/step - loss

4/4 [==============================] - 0s 667us/step - loss: 0.0193 - mse: 0.0193
Epoch 175/300
4/4 [==============================] - 0s 667us/step - loss: 0.0194 - mse: 0.0194
Epoch 176/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0194 - mse: 0.0194
Epoch 177/300
4/4 [==============================] - 0s 667us/step - loss: 0.0194 - mse: 0.0194
Epoch 178/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0194 - mse: 0.0194
Epoch 179/300
4/4 [==============================] - 0s 667us/step - loss: 0.0190 - mse: 0.0190
Epoch 180/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0192 - mse: 0.0192
Epoch 181/300
4/4 [==============================] - 0s 1000us/step - loss: 0.0194 - mse: 0.0194
Epoch 182/300
4/4 [==============================] - 0s 667us/step - loss: 0.0193 - mse: 0.0193
Epoch 183/300
4/4 [==============================] - 0s 667us/step - loss: 0.0193 - mse: 0.0193
Epoch 184/300
4/4 [==============================] - 0s 667

4/4 [==============================] - 0s 667us/step - loss: 0.0184 - mse: 0.0184
Epoch 261/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0183 - mse: 0.0183
Epoch 262/300
4/4 [==============================] - 0s 667us/step - loss: 0.0186 - mse: 0.0186
Epoch 263/300
4/4 [==============================] - 0s 667us/step - loss: 0.0181 - mse: 0.0181
Epoch 264/300
4/4 [==============================] - 0s 667us/step - loss: 0.0191 - mse: 0.0191
Epoch 265/300
4/4 [==============================] - 0s 667us/step - loss: 0.0180 - mse: 0.0180
Epoch 266/300
4/4 [==============================] - 0s 667us/step - loss: 0.0181 - mse: 0.0181
Epoch 267/300
4/4 [==============================] - 0s 667us/step - loss: 0.0180 - mse: 0.0180
Epoch 268/300
4/4 [==============================] - 0s 667us/step - loss: 0.0180 - mse: 0.0180
Epoch 269/300
4/4 [==============================] - 0s 1ms/step - loss: 0.0182 - mse: 0.0182
Epoch 270/300
4/4 [==============================] - 0s 66

In [86]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2score = r2_score(y_test, y_pred)

1/1 [==============================] - 0s 41ms/step


In [87]:
print("RMSE:", rmse)
print("R2_score:", r2score)

RMSE: 0.13498741416338297
R2_score: 0.7214688624294318
